# **Flooding in Italy - working code (R)**

In [1]:
source("../methods_extRemes.r")

prep_window <- function(rc = c(1,1), w = 4, h = 4) { options(repr.plot.width = rc[2]*w, repr.plot.height = rc[1]*h, repr.plot.res = 200); par(mfrow = rc, pch = 20) }
load_ts <- function(fnm, col.names) { read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = col.names) }

# parameters for confidence intervals
rp <- 100
nsamp <- 1000

In [2]:
# support function to give required results
mdl_ests <- function(mdl, cov, cov_cf, ev, rp = NA) {

    pars <- mdl$results$par
    disp <- unname(pars["sigma0"] / pars["mu0"])

    if(is.na(rp)) rp <- return_period(mdl, value = ev, covariate = cov, lower = F)
    if(is.finite(rp)) {
        rp_cf <- return_period(mdl, value = ev, covariate = cov_cf)
        rl_cf <- unname(return_level(mdl, rp = rp, covariate = cov_cf))

        c(pars,
          "disp" = disp, 
          "event_magnitude" = ev, 
          "return_period" = rp, 
          "PR" = rp_cf / rp, 
          "dI_abs" = ev - rl_cf, 
          "dI_rel" = (ev - rl_cf) / rl_cf * 100)
    } else {
        return(rep(NA, 10))
    }
}

In [3]:
# wrapper function to get bootstrapped confidence intervals for spreadsheet
boot_ci <- function(mdl, cov, cov_cf, ev = NA, rp = NA, seed = 42, nsamp = 500, dp = 5) {
    
    # get best estimate from the observed data
    if(is.na(ev)) ev  <- mdl$x[length(mdl$x)]
    mdl_res <- mdl_ests(mdl, cov, cov_cf, ev, rp = rp)
    mdl_df <- mdl$cov.data
    
    # get bootstrap sample
    set.seed(seed)    
    boot_res <- sapply(1:nsamp, function(i) {
        boot_df <- mdl_df[sample(1:nrow(mdl_df), nrow(mdl_df), replace = T),]
        tryCatch({
            boot_mdl <- fevd_fixeddisp("rx21day", "gmst", boot_df, solver = "Nelder-Mead")
            mdl_ests(boot_mdl, cov, cov_cf, ev, rp = rp)
        },
        error = function(cond) {return(rep(NA, 10))})
    })
    boot_qq <- t(rbind("bestimate" = mdl_res, apply(boot_res, 1, quantile, c(0.025, 0.975), na.rm = T)))
    if(!is.na(dp)) boot_qq <- round(boot_qq, dp)
    return(boot_qq)
}

In [4]:
model_results <- function(df, rp = 10, cov_pres, cov_pi, cov_fut, nsamp = 5, seed = 42) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    # fit models
    mdl_eval <- fevd_fixeddisp("rx21day", "gmst", df[df$year >= 1979 & df$year <= 2023,], solver = "Nelder-Mead")
    mdl_attr <- fevd_fixeddisp("rx21day", "gmst", df[df$year <= 2023,], solver = "Nelder-Mead")
    mdl_proj <- fevd_fixeddisp("rx21day", "gmst", df[df$year <= 2050,], solver = "Nelder-Mead")
    
    # get return level to use for analysis
    event_rl <- unname(return_level(mdl_attr, rp = rp, covariate = cov_pres))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## Bootstrap each set of model results

    ci_eval <- boot_ci(mdl_eval, cov = cov_pres, cov_cf = cov_pi, ev = event_rl, rp = rp, nsamp = nsamp)[c("disp", "shape"),]
    ci_attr <- boot_ci(mdl_attr, cov = cov_pres, cov_cf = cov_pi, ev = event_rl, rp = rp, nsamp = nsamp)[c("PR", "dI_rel"),]
    ci_proj <- boot_ci(mdl_proj, cov = cov_pres, cov_cf = cov_fut, ev = event_rl, rp = rp, nsamp = nsamp)[c("PR", "dI_rel"),]
                   
    # invert future projections
    ci_proj["PR",] <- 1/ci_proj["PR",c(1,3,2)]
    ci_proj["dI_rel",] <- -ci_proj["dI_rel", c(1,3,2)]
    
    # flatten & rename
    ci_eval <- unlist(lapply(rownames(ci_eval), function(cnm) setNames(ci_eval[cnm,], paste("eval", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
    ci_attr <- unlist(lapply(rownames(ci_attr), function(cnm) setNames(ci_attr[cnm,], paste("attr", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
    ci_proj <- unlist(lapply(rownames(ci_proj), function(cnm) setNames(ci_proj[cnm,], paste("proj", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
                         
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    t(data.frame(c(ci_eval, "rp_value" = event_rl, ci_attr, ci_proj)))     
}

---
## **Trend analysis - observations**

In [4]:
gmst <- load_ts("ts/Italy-floods_gmst-smoothed.dat", col.names = c("year", "gmst"))
gmst_2023 <- gmst[gmst$year == 2023, "gmst"]
gmst_cf <- gmst_2023 - 1.2

In [6]:
# MSWEP
df <- merge(gmst, load_ts("ts/Italy-floods_rx21day-amj_mswep.dat", col.names = c("year", "rx21day")))
ns_mdl <- fevd_fixeddisp("rx21day", "gmst", df, solver = "Nelder-Mead")
boot_res <- boot_ci(ns_mdl, gmst_2023, gmst_cf, nsamp = 1000)
write.csv(t(boot_res), "res/res_Italy-floods_rx21day-amj_mswep.csv")

In [7]:
# ERA5
df <- merge(gmst, load_ts("ts/Italy-floods_rx21day-amj_era5.dat", col.names = c("year", "rx21day")))
ns_mdl <- fevd_fixeddisp("rx21day", "gmst", df, solver = "Nelder-Mead")
boot_res <- boot_ci(ns_mdl, gmst_2023, gmst_cf, nsamp = 1000)
write.csv(t(boot_res), "res/res_Italy-floods_rx21day-amj_era5.csv")

## **Trend analysis - CORDEX**

In [6]:
# loop over all available models, get results for each
invisible(sapply(list.files("ts", pattern = "rx21day-amj_EUR-11", full.names = T), function(fnm) {

    res_fnm <- paste0("res/res_",gsub("ts/", "", gsub(".dat", "", fnm)), ".csv")
    print(fnm)
    if(!file.exists(res_fnm)) {
        
        # load data   
        gmst_fnm <- list.files("ts", paste0("smoothed-gsat_", strsplit(fnm, "_")[[1]][4], "_rcp85_", strsplit(fnm, "_")[[1]][5], ".dat"), full.names = T)
        df <- merge(load_ts(gmst_fnm, col.names = c("year", "gmst")), 
                    load_ts(fnm, col.names = c("year", "rx21day")))
        
        gmst_2023 <- df[df$year == 2023, "gmst"]
        gmst_pi <- gmst_2023 - 1.2
        gmst_fut <- gmst_2023 + 0.8

        res_df <- model_results(df, rp = rp, gmst_2023, gmst_pi, gmst_fut, nsamp = nsamp)
        rownames(res_df) <- gsub(".dat", "", paste0(strsplit(fnm, "_")[[1]][4:6], collapse = "_"))

        write.csv(res_df, res_fnm)
    }
}))

[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_ALADIN53.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_ALADIN63.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_ALARO-0.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_CCLM4-8-17.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_crCLIM-v1-1.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_HadREM3-GA7-05.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_RACMO22E.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_RCA4.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_RegCM4-6.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_REMO2015.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_CNRM-CM5_r1_WRF381P.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_EC-EARTH_r1_crCLIM-v1-1.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_EC-EARTH_r1_RACMO22E.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_EC-EARTH_r1_RCA4.dat"
[1] "ts/Italy-floods_rx21day-amj_EUR-11_EC-EARTH_r12_CCLM4-8-17.dat"
[1]

In [8]:
# compile into single file 
res <- do.call("rbind", sapply(list.files("res", pattern = "EUR-11", full.names = T), read.csv, row.names = 1, simplify = F))
rownames(res) <- sapply(rownames(res), function(rnm) gsub(".csv", "", paste0(strsplit(rnm, "_")[[1]][5:7], collapse = "_")))
write.csv(res, "Italy-floods_cordex-results.csv")

## **Trend analysis - UKCP**

In [5]:
# loop over all available models, get results for each
invisible(sapply(list.files("ts", pattern = "rx21day-amj_UKCP18", full.names = T), function(fnm) {
    
    ens <- gsub(".dat", "", gsub(".+_", "", fnm))
    res_fnm <- paste0("res/res_",gsub("ts/", "", gsub(".dat", "", fnm)), ".csv")
    print(fnm)
    if(!file.exists(res_fnm)) {
        
        # load data   
        gmst_fnm <- list.files("ts", paste0("gsat-aw_sm_", ens), full.names = T)
        df <- merge(load_ts(gmst_fnm, col.names = c("year", "gmst")), 
                    load_ts(fnm, col.names = c("year", "rx21day")))
        
        gmst_2023 <- df[df$year == 2023, "gmst"]
        gmst_pi <- gmst_2023 - 1.2
        gmst_fut <- gmst_2023 + 0.8
        
        res_df <- model_results(df, rp = rp, gmst_2023, gmst_pi, gmst_fut, nsamp = nsamp)
        rownames(res_df) <- ens

        write.csv(res_df, res_fnm)
    }
}))

[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_01.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_04.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_05.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_06.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_07.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_08.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_09.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_10.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_11.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_12.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_13.dat"
[1] "ts/Italy-floods_rx21day-amj_UKCP18-12km_15.dat"


In [14]:
# compile into single file 
res <- do.call("rbind", sapply(list.files("res", pattern = "UKCP18", full.names = T), read.csv, row.names = 1, simplify = F))
rownames(res) <- sapply(rownames(res), function(rnm) gsub(".csv", "", gsub(".+_", "", rnm)))
write.csv(res, "Italy-floods_ukcp18-results.csv")

---
## **Trends in frequency of extreme short-duration precipitation**